In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ------- Import Libs ---------
from datetime import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import math
import pytz

import warnings
import re

import plotly.express as px

alt.renderers.enable("altair_viewer")

# Disbale the max row limit for altair datasets.
alt.data_transformers.disable_max_rows()

In [ ]:
# Set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

print(CWD)

In [ ]:
# This function does the same as above but orders the dataframe correct
# I should also make the folder path a variable so that I am perform some quick stats analysis on it
def folder_data_load_sorted(folder_path):
    # This is a list of all of the filenames
    files = os.listdir(folder_path)

    # Read all Excel files into a list of dataframes
    dataframes = []

    for filename in files:
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            print(file_path)

            # Extract the reference date from the filename
            reference_date = filename.split("_")[0]

            df = pd.read_excel(file_path, engine="openpyxl")

            # create new columns in the data for date specific
            df["Year"] = reference_date.split("-")[0]
            df["Month"] = reference_date.split("-")[1]
            df["Day"] = reference_date.split("-")[2]

            # Convert the 'Time' column to datetime.time objects
            df["Time2"] = pd.to_datetime(df["Time"], format="%H:%M:%S").dt.time
            df["Time2"] = df["Time2"].astype(str)

            df["Date"] = pd.to_datetime(df["Year"] + df["Month"] + df["Day"], format="%Y-%m-%d")

            # Convert the 'Time' column to datetime objects using the extracted reference date
            df["Datetime"] = pd.to_datetime(
                reference_date + " " + df["Time2"], format="%Y-%m-%d %H:%M:%S"
            )

            # This returns a tuple
            dataframes.append((reference_date, df))

    dataframes.sort(key=lambda x: x[0])

    sorted_dataframes = [df for _, df in dataframes]

    return sorted_dataframes

In [ ]:
# Best to create a way to look at all of the data now rather then just one specific year.
def load_data_from_all_years(parent_folder_path):
    # Initialize an empty list to store the dataframes
    all_dataframes = []

    # Loop over each year's folder and call the folder_data_load_sorted function
    for year in range(2018, 2024):
        folder_path = os.path.join(parent_folder_path, str(year))
        dataframes = folder_data_load_sorted(folder_path)
        all_dataframes.extend(dataframes)

    return all_dataframes

In [ ]:
# A function to load a single excel file
def read_excel_file(file_path):
    df = pd.read_excel(file_path, engine="openpyxl")
    return df

In [ ]:
# ---------- 15 min data ------------
def load_all_15min(parent_folder_path):

    all_dataframes = load_data_from_all_years(parent_folder_path)
    all_combined_df = pd.concat(all_dataframes, ignore_index=True)

    # Sort the DataFrame based on the 'Datetime' column
    all_combined_df_sort = all_combined_df.sort_values(by="Datetime")

    return all_combined_df_sort

In [ ]:
# define the function to transform the dates to fix the bug
def transform_date(date_str):
    date_format = "%Y-%m-%d %H:%M:%S"
    date_obj = datetime.strptime(date_str, date_format)
    new_date = date_obj.replace(hour=date_obj.minute, minute=date_obj.second, second=0)
    return new_date.strftime(date_format)

In [ ]:
# --------- 15 min data opperation ----------
# 1. Load the data into a single list
parent_folder_path = "./data/island_A/clean_data/15-min-PV/"
data_15min = load_all_15min(parent_folder_path)

print("-------- COMPLETE 1 -------")

# This code here is if you want to just load a single year of 15 min data
#     folder_path = "./data/island_A/15-min-PV/2019/"
#     data_15min = folder_data_load_sorted(folder_path)

In [ ]:
# 2. Fix the bugs with datetime interp
# define the bug_start and bug_end dates + May need to enter new dates and check format
bug_start = datetime.strptime("2018-12-19 23:45:00", "%Y-%m-%d %H:%M:%S")
bug_end = datetime.strptime("2019-10-05 00:00:00", "%Y-%m-%d %H:%M:%S")

data_15min["Datetime"] = pd.to_datetime(data_15min["Datetime"], format="%Y-%m-%d %H:%M:%S")

# apply the transformation to the appropriate rows
mask = (data_15min["Datetime"] > bug_start) & (data_15min["Datetime"] < bug_end)
data_15min.loc[mask, "Datetime"] = data_15min.loc[mask, "Datetime"].apply(
    lambda x: transform_date(x.strftime("%Y-%m-%d %H:%M:%S"))
)
print("-------- COMPLETE 2 -------")


print("-------- COMPLETE 5 -------")

In [ ]:
# 5. Save the data as a csv
data_15min.to_csv("data/island_A/exported/data_15min_clean.csv", index=False)

data_15min_conv = data_15min
print("-------- COMPLETE 6 -------")

In [ ]:
data_15min.head(5)

In [ ]:
# 3. Apply UTC Convention
# View all timezones using the code below
"""
>>> import pytz
>>> pytz.all_timezones
"""


def convert_to_utc(df, source_timezone):
    # Ensure the DataFrame has a DatetimeIndex
    #     if not isinstance(df.index, pd.DatetimeIndex):
    #         raise ValueError("The DataFrame must have a DatetimeIndex.")

    # Create timezone objects for source and target (UTC) timezones
    source_tz = pytz.timezone(source_timezone)
    target_tz = pytz.UTC

    # Convert the "datetime" column to a DatetimeIndex
    # df['Datetime'] = pd.to_datetime(df['Datetime'])

    if not isinstance(df.index, pd.DatetimeIndex):
        # Convert the "datetime" column to a DatetimeIndex
        df["Datetime"] = pd.to_datetime(df["Datetime"])
        df.set_index("Datetime", inplace=True)

    # df.set_index('Datetime', inplace=True)

    # Localize the DatetimeIndex to the source timezone, handling ambiguous and non-existent times
    df_source_tz = df.index.tz_localize(source_tz, ambiguous="NaT", nonexistent="NaT")

    # Convert the DatetimeIndex to the target timezone (UTC)
    df_utc = df_source_tz.tz_convert(target_tz)

    # Set the DatetimeIndex as a column in the DataFrame
    df["datetimeUTC"] = df_utc

    df.set_index("datetimeUTC", inplace=True)

    # to get rid of the time zone different impliment:
    # Format the DatetimeIndex without the timezone offset
    # df_utc.index = df_utc.index.strftime('%Y-%m-%d %H:%M:%S')

    return df


# orginally got this error: NonExistentTimeError: 2019-03-31 02:00:00, so they dont adjust for daylight savings.


source_timezone = "Europe/Malta"  # Replace with the desired timezone
data_15min_utc = convert_to_utc(data_15min_conv, source_timezone)


# will need to createa a lamda/function to go through all of the 15min data to convert it
# to the correct UTC. It would be worth making this into a function so it can be used by the hourly.
# Want it so that i just have to pass a dataframe that has a Datetime funciton into it.

print("-------- COMPLETE 3 -------")

In [ ]:
data_15min_utc

In [ ]:
def drop_unused_15min(df):

    df = df.drop(
        ["Time", "Year", "Month", "Day", "Time2", "Date", "Hour number", "Hourly Average"], 1
    )

    df = df.reset_index()
    df["datetimeUTC"] = pd.to_datetime(df["datetimeUTC"])

    return df


data_15min_utc_drop = drop_unused_15min(data_15min_utc)

In [ ]:
data_15min_utc_drop

In [ ]:
# 8. Convert into Xarray and then NetCDF format
# Converting to an Xarray
def pdtocdf(df, file_name):

    data_array = xr.Dataset(df)

    data_array = data_array.set_coords("datetimeUTC").swap_dims({"dim_0": "datetimeUTC"})

    data_array = data_array.drop("dim_0")

    save_directory = "./data/island_A/exported/"
    file_end = ".nc"
    full_file_path = save_directory + file_name + file_end

    # Save the DataArray as a NetCDF file
    data_array.to_netcdf(full_file_path)

    print(f"File saved at: {full_file_path}")

    return data_array


data_15min_xarray = pdtocdf(data_15min_utc_drop, "data_15min_FINv2")

In [ ]:
data_15min_xarray

In [ ]:
# 6. Create a new rolling sum of the 15min data
data_15min_resample = data_15min

# data_15min_rolling['15-Minute Output MWh'].rolling(4).sum()
# data_15min_rolling['Power at point in time MW'].rolling(4).sum()


# Assuming your DataFrame has a 'timestamp' column and columns 'A' and 'B'
data_15min_resample["Datetime"] = pd.to_datetime(data_15min_resample["Datetime"])
data_15min_resample.set_index("Datetime", inplace=True)

# Resample column 'B' to hourly frequency and sum the values
df_B_hourly_sum_MWh = data_15min_resample["15-Minute Output MWh"].resample("H").sum()
df_B_hourly_sum_MW = data_15min_resample["Power at point in time MW"].resample("H").sum()

In [ ]:
# 7. Save the data as a csv
df_B_hourly_sum_MWh.to_csv("data/island_A/exported/15mintohour_MWh.csv", index=True)

# 7. Save the data as a csv
df_B_hourly_sum_MW.to_csv("data/island_A/exported/15mintohour_MW.csv", index=True)

print("-------- COMPLETE 7 -------")

In [ ]:
# 7.5 Complete the normalisation.

In [ ]:
# 7. Save the data as a csv
data_15min_rolling.to_csv("data/island_A/exported/data_15min_rolling.csv", index=False)
print("-------- COMPLETE 8 -------")

In [ ]:
def folder_data_load_sorted_h(folder_path):
    # This is a list of all of the filenames
    files = os.listdir(folder_path)

    # Read all Excel files into a list of dataframes
    dataframes = []

    for filename in files:
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)

            df = pd.read_excel(file_path, engine="openpyxl")

            # This returns a tuple
            dataframes.append((reference_date, df))

    dataframes.sort(key=lambda x: x[0])

    sorted_dataframes = [df for _, df in dataframes]

    return sorted_dataframes

In [ ]:
# Best to create a way to look at all of the data now rather then just one specific year.
def load_data_from_all_years_h(parent_folder_path_hourly):
    # Initialize an empty list to store the dataframes
    all_dataframes = []

    # Loop over each year's folder and call the folder_data_load_sorted function

    # folder_path = os.path.join(parent_folder_path, ,str(year))
    dataframes = folder_data_load_sorted_h(parent_folder_path_hourly)
    all_dataframes.extend(dataframes)

    return all_dataframes

In [ ]:
# ---------- Hourly data -------------
def load_all_hourly(parent_folder_path):

    # all_dataframes = load_data_from_all_years_h(parent_folder_path)
    # all_combined_df = pd.concat(all_dataframes, ignore_index=True)

    # This is a list of all of the filenames
    files = os.listdir(parent_folder_path)

    # Read all Excel files into a list of dataframes
    dataframes = []

    for filename in files:
        if filename.endswith(".xlsx"):
            file_path = os.path.join(parent_folder_path, filename)

            df = pd.read_excel(file_path, engine="openpyxl")

            # This returns a tuple
            dataframes.append(df)

    # dataframes.sort(key=lambda x: x[0])

    # sorted_dataframes = [df for _, df in dataframes]

    all_combined_df = pd.concat(dataframes, ignore_index=True)

    # Sort the DataFrame based on the 'Datetime' column
    all_combined_df_sort = all_combined_df.sort_values(by="Date")

    return all_combined_df_sort

    # Sort the DataFrame based on the 'Datetime' column
    # all_combined_df_sort = all_combined_df.sort_values(by="Datetime")

    # return all_combined_df_sort

In [ ]:
# 1. Load the data into a single list
parent_folder_path_hourly = "./data/island_A/clean_data/hourly-PV/"
hourly_data_raw = load_all_hourly(parent_folder_path_hourly)
print("--------- COMPLETE 1 --------")

In [ ]:
hourly_data_raw.iloc[1000]

In [ ]:
"""
# 1. Load the data - Single excel file 
folder_path = "./data/island_A/Hourly-PV/format_test/HourlyPVgeneratedUnits_2018_test.xlsx"
# folder_path = './data/island_A/15-min-PV/2019/2019-01-03_PVMalta.xlsx'
hourly_data_raw = read_excel_file(folder_path)

"""

In [ ]:
# 2. Convert into usable format (Transpose of hours)
def transpose_data(df):

    # Convert column names to integers
    hour_columns = [col for col in df.columns if str(col).isdigit()]

    # melt the data
    # XXX Need to retain other information, edit this
    melted = df.melt(
        id_vars=[
            "Date",
            " Total Max Capacity of Read Meters/KW",
            "Total Max Capacity",
            "Number of Read Meters",
            "Total Number of Meters",
        ],
        value_vars=hour_columns,
        var_name="Hour",
    )

    melted = melted.dropna()

    melted["Date"] = pd.to_datetime(melted["Date"])
    melted["Hour"] = pd.to_timedelta(melted["Hour"], unit="h")

    melted["Datetime"] = melted["Date"] + melted["Hour"]

    # Sort the DataFrame based on the 'Datetime' column
    melted_sorted = melted.sort_values(by="Datetime")

    melted_sorted.rename(columns={"value": "Hourly PV Generated Units"}, inplace=True)

    return melted_sorted


hourly_data = transpose_data(hourly_data_raw)

print("--------- COMPLETE 2 --------")

In [ ]:
hourly_data

In [ ]:
# 3. Save the file
hourly_data.to_csv("data/island_A/exported/hourly_data_clean.csv", index=False)

In [ ]:
# 4. Apply UTC conversion
source_timezone = "Europe/Malta"  # Replace with the desired timezone
data_hourly_utc = convert_to_utc(hourly_data, source_timezone)

In [ ]:
data_hourly_utc

In [ ]:
# 5. Drop unused information
def drop_unused_hourly(df):

    df = df.drop(["Date", "Hour"], 1)

    df = df.reset_index()
    df["datetimeUTC"] = pd.to_datetime(df["datetimeUTC"])

    return df


data_hourly_utc_drop = drop_unused_hourly(data_hourly_utc)

In [ ]:
data_hourly_utc_drop

In [ ]:
# 5. Convert into Xarray and then NetCDF format
data_hourly_xarray = pdtocdf(data_hourly_utc_drop, "data_hourly_FIN")

In [ ]:
data_hourly_xarray

In [ ]:

%history -g
%save -r backuplog 1-999999


In [ ]:
%history -g -f session.txt